## PROJECT-2: PART-1 - Algorithmic Trading: Moving Average

---

#### Importing libraries and dependencies

In [5]:
# Import libraries and dependencies
import numpy as np
import pandas as pd
import hvplot.pandas
from pathlib import Path
import matplotlib.pyplot as plt
import warnings
warnings.simplefilter(action="ignore")

#### Function to generate trade evaluation metrics 

In [2]:
# Function to Read Source CSV

def trade_evaluation(stock_name, short_window, long_window, share_size, initial_capital):
    
    # Read the CSV located at the file path into a Pandas DataFrame
    filepath = Path('Resources/'+str(stock_name)+'.csv')
    
    # Set the `Date` column as the index and auto-format the datetime string
    stock_csv = pd.read_csv(filepath, parse_dates=True, infer_datetime_format=True)
    stock_csv = stock_csv.dropna()
    
    # Set short and long window
    str_short="SMA"+str(short_window)
    str_long="SMA"+str(long_window)
    
    # Grab just the `date` and `close` from the dataset
    signals_df = stock_csv.loc[:, ["Date", "Close"]].copy()

    # Set the `date` column as the index
    signals_df = signals_df.set_index("Date", drop=True)

    # Generate the short and long moving averages
    signals_df[str_short] = signals_df["Close"].rolling(window = short_window).mean()
    signals_df[str_long] = signals_df["Close"].rolling(window = long_window).mean()
    signals_df["Signal"] = 0.0

    # Generate the trading signal 0 or 1,
    # where 0 is when the short window is under the long window, and
    # where 1 is when the short window is higher (or crosses over) the long window
    signals_df["Signal"][short_window:] = np.where(
        signals_df[str_short][short_window:] > signals_df[str_long][short_window:], 1.0, 0.0)

    # Calculate the points in time at which a position should be taken, 1 or -1
    signals_df["Entry/Exit"] = signals_df["Signal"].diff()
    
    # Take a 500 share position where the dual moving average crossover is 1 (SMA-short window > SMA-long window)
    signals_df["Position"] = share_size * signals_df["Signal"]

    # Find the points in time where a 500 share position is bought or sold
    signals_df["Entry/Exit Position"] = signals_df["Position"].diff()

    # Multiply share price by entry/exit positions and get the cumulatively sum
    signals_df["Portfolio Holdings"] = (
        signals_df["Close"] * signals_df["Entry/Exit Position"].cumsum())

    # Subtract the initial capital by the portfolio holdings to get the amount of liquid cash in the portfolio
    signals_df["Portfolio Cash"] = (
        initial_capital - (signals_df["Close"] * signals_df["Entry/Exit Position"]).cumsum())

    # Get the total portfolio value by adding the cash amount by the portfolio holdings (or investments)
    signals_df["Portfolio Total"] = (
        signals_df["Portfolio Cash"] + signals_df["Portfolio Holdings"])

    # Calculate the portfolio daily returns
    signals_df["Portfolio Daily Returns"] = signals_df["Portfolio Total"].pct_change()

    # Calculate the cumulative returns
    signals_df["Portfolio Cumulative Returns"] = (1 + signals_df["Portfolio Daily Returns"]).cumprod() - 1
    
    # Initialize trade evaluation DataFrame with columns
    trade_evaluation_df = pd.DataFrame(columns=['Stock','Entry Date','Exit Date','Shares',
                                                'Entry Share Price','Exit Share Price','Entry Portfolio Holding',
                                                'Exit Portfolio Holding','Profit/Loss'])
    
    # Initialize iterative variables
    entry_date = ''
    exit_date = ''
    entry_portfolio_holding = 0
    exit_portfolio_holding = 0
    share_size = 0
    entry_share_price = 0
    exit_share_price = 0

    # Loop through signal DataFrame
    # If `Entry/Exit` is 1, set entry trade metrics
    # Else if `Entry/Exit` is -1, set exit trade metrics and calculate profit,
    # Then append the record to the trade evaluation DataFrame
    for index, row in signals_df.iterrows():
        if row['Entry/Exit'] == 1:
            entry_date = index
            entry_portfolio_holding = abs(row['Portfolio Holdings'])
            share_size = row['Entry/Exit Position']
            entry_share_price = row['Close']

        elif row['Entry/Exit'] == -1:
            exit_date = index
            exit_portfolio_holding = abs(row['Close'] * row['Entry/Exit Position'])
            exit_share_price = row['Close']
            profit_loss =  exit_portfolio_holding - entry_portfolio_holding
            trade_evaluation_df = trade_evaluation_df.append(
                {
                    'Stock': stock_name,
                    'Entry Date': entry_date,
                    'Exit Date': exit_date,
                    'Shares': share_size,
                    'Entry Share Price': entry_share_price,
                    'Exit Share Price': exit_share_price,
                    'Entry Portfolio Holding': entry_portfolio_holding,
                    'Exit Portfolio Holding': exit_portfolio_holding,
                    'Profit/Loss': profit_loss
                },
                ignore_index=True)
  
    return trade_evaluation_df

#### Building a composite dataframe of profitable trade evaluation metrics

In [3]:
# Build empty composite dataframe
composite_trade_evaluation_df = pd.DataFrame(
        columns=[
            'Stock', 
            'Entry Date', 
            'Exit Date', 
            'Shares', 
            'Entry Share Price', 
            'Exit Share Price', 
            'Entry Portfolio Holding', 
            'Exit Portfolio Holding', 
            'Profit/Loss'])

#### Input parameters

In [4]:
# Input Parameters

# Stock Source File Names
stock_name = ['AAPL','BTC-USD','JPM','MA','MRNA','TNX','V','AMZN','FB','Gold','GOOG','MSFT','Silver','TSLA','TWTR']

# Set the short window and long trade windows
short_window = [9, 21, 9, 50, 21, 50]
long_window = [21, 50, 50, 100, 100, 200]

# Set initial capital
initial_capital = float(100000)

# Set the share size
share_size = 500

# Building a composite dataframe
for i in range(len(stock_name)):
    for j in range(len(short_window)):
        temp = trade_evaluation(stock_name[i], short_window[j], long_window[j], share_size, initial_capital)
        if len(temp['Exit Portfolio Holding'])>0 and temp['Exit Portfolio Holding'][(len(temp['Exit Portfolio Holding']))-1]>temp['Exit Portfolio Holding'][0]:
                composite_trade_evaluation_df = composite_trade_evaluation_df.append(temp)

composite_trade_evaluation_df = composite_trade_evaluation_df.reset_index()
composite_trade_evaluation_df

,index,Stock,Entry Date,Exit Date,Shares,Entry Share Price,Exit Share Price,Entry Portfolio Holding,Exit Portfolio Holding,Profit/Loss
0,0,AAPL,2/5/2019,5/14/2019,500.0,43.544998,47.165001,21772.4990,23582.5005,1810.0015
1,1,AAPL,6/12/2019,8/7/2019,500.0,48.547501,49.759998,24273.7505,24879.9990,606.2485
2,2,AAPL,8/21/2019,2/25/2020,500.0,53.160000,72.019997,26580.0000,36009.9985,9429.9985
3,3,AAPL,4/8/2020,9/14/2020,500.0,66.517502,115.360001,33258.7510,57680.0005,24421.2495
4,4,AAPL,10/6/2020,10/28/2020,500.0,113.160004,111.199997,56580.0020,55599.9985,-980.0035
...,...,...,...,...,...,...,...,...,...,...
331,0,TWTR,3/25/19,6/6/19,500.0,32.590000,36.590000,16295.0000,18295.0000,2000.0000
332,1,TWTR,7/15/19,10/2/19,500.0,38.680000,39.700000,19340.0000,19850.0000,510.0000
333,2,TWTR,12/26/19,3/9/20,500.0,32.630000,32.460000,16315.0000,16230.0000,-85.0000
334,3,TWTR,5/4/20,7/2/20,500.0,28.230000,30.870000,14115.0000,15435.0000,1320.0000


#### Note: "composite_trade_evaluation_df" dataframe is sourced as "_COMPOSITE2.csv" to calculate NLP stock sentiment scores in "ALGO_NLP_DEV_4_FINAL.ipynb"